# Lab 5: Land Surface Temperature using CORINE-based Emissivity

## 🎯 Objectives
In this exercise, you will:
- Select a cloud-free Landsat 8 images from 2013 and 2023 (or different if you're sure that you'll spot a difference in land cover)
- Calculate Brightness Temperature (TB) from Band 10.
- Load CORINE Land Cover data and assign emissivity values to each land cover class.
- Use the Planck-based formula to calculate Land Surface Temperature (LST).
- Visualize and interpret the results.

## Step 1: Define Area of Interest (AOI)
- Use coordinates around Reduta street in Kraków.
- Create a polygon or rectangle using `ee.Geometry.Polygon`.

In [3]:
# Define AOI here
import ee
#ee.Authenticate()
#ee.Initialize(project='ee-piotrkrajewski')

aoi = ee.Geometry.Rectangle([21.449994, 49.733740, 21.494039, 49.759334 ])  # Example coordinates for Jasło, Poland
print(aoi.getInfo())


{'type': 'Polygon', 'coordinates': [[[21.449994, 49.73374], [21.494039, 49.73374], [21.494039, 49.759334], [21.449994, 49.759334], [21.449994, 49.73374]]]}


## Step 2: Load Landsat 8 imagery for the dates you've picked
- Filter for low cloud cover (< 20%)
- Select Band 10 and convert to TB using: `TB = ST_B10 * 0.00341802 + 149.0`

In [ ]:
# Load image and calculate TB
import geemap 
start_2013 = ee.Date('2013-08-01') 
end_2013 = ee.Date('2013-08-30')
start_2024 = ee.Date('2023-08-01') # one week of difference
end_2024 = ee.Date('2023-08-30')

image_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(aoi).filterDate(start_2013, end_2013).filterMetadata('CLOUD_COVER', 'less_than', 20).first()
image_2024 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(aoi).filterDate(start_2024, end_2024).filterMetadata('CLOUD_COVER', 'less_than', 20).first()
image_2013 = image_2013.clip(aoi)
image_2024 = image_2024.clip(aoi)
dateofimage_2013 = image_2013.date().format('YYYY-MM-dd').getInfo()
dateofimage_2024 = image_2024.date().format('YYYY-MM-dd').getInfo()
print(dateofimage_2013)
print(dateofimage_2024)
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 5000,
    'max': 15000,
    'gamma': 1.3
}
center = aoi.centroid(10).coordinates().reverse().getInfo()

Map = geemap.Map(center=center, zoom=15)

Map.addLayer(image_2013, vis_params, 'Landsat 2013-08-09')
Map.addLayer(image_2024, vis_params, '2023-08-13')

Map

bt_2013 = image_2013.select('ST_B10').multiply(0.00341802).add(149.0)
bt_2024 = image_2024.select('ST_B10').multiply(0.00341802).add(149.0)


thermal_vis = {
    'min': 290,
    'max': 320,
    'palette': ['blue', 'green',  'red']
}

center = aoi.centroid(10).coordinates().reverse().getInfo()
Map = geemap.Map(center=center, zoom=15)
Map.addLayer(bt_2013, thermal_vis, '2013 Brightness Temp')
Map.addLayer(bt_2024, thermal_vis, '2024 Brightness Temp')
Map


2013-08-09
2023-08-13


Map(center=[49.74653796191832, 21.472016499992947], controls=(WidgetControl(options=['position', 'transparent_…

## Step 3: Load CORINE Land Cover data
- Use dataset `COPERNICUS/CORINE/V20/100m/2018`
- Clip it to your AOI

In [28]:
# Load CORINE dataset and clip to AOI
corine = ee.Image('COPERNICUS/CORINE/V20/100m/2018').clip(aoi)

# Add CORINE layer to the map for visualization
corine_vis = {
    'min': 111,
    'max': 999,
    'palette': ['006400', 'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4', 'f0f0f0', '0064c8', '0096a0', '00cf75']
}
Map.addLayer(corine, corine_vis, 'CORINE Land Cover 2018')
Map

Map(bottom=713884.0, center=[49.74048251739346, 21.487756448199082], controls=(WidgetControl(options=['positio…

## Step 4: Assign emissivity to CORINE classes
- Use a dictionary for classes
- Use `remap()` and optionally a default value

In [29]:
# Create emissivity image

emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}
emmisivity_image = ee.Image(0).rename('emissivity')
for code, emissivity in emissivity_dict.items():
    mask = corine.eq(code)
    emmisivity_image = emmisivity_image.where(mask, emissivity)

## Step 5: Calculate LST using the formula:
$$
LST = \frac{T_B}{1 + \left( \frac{\lambda \cdot T_B}{c_2} \right) \cdot \ln(\varepsilon)}
$$
- λ = 10.8 µm
- c₂ = 14388 µm·K

In [ ]:
# Calculate LST

lambda_val = 10.8e-6  
c2 = 1.4388e-2  

lst_2013 = bt_2013.expression(
    'TB / (1 + (lambda * TB / c2) * log(emissivity))',
    {
        'TB': bt_2013,
        'lambda': lambda_val,
        'c2': c2,
        'emissivity': emmisivity_image
    }
)

lst_2024 = bt_2024.expression(
    'TB / (1 + (lambda * TB / c2) * log(emissivity))',
    {
        'TB': bt_2024,
        'lambda': lambda_val,
        'c2': c2,
        'emissivity': emmisivity_image
    }
)



Map(bottom=713884.0, center=[49.74048251739346, 21.487756448199082], controls=(WidgetControl(options=['positio…

## Step 6: Visualize the LST result
- Use palette: `['blue', 'yellow', 'red']`
- Suggested range: `min=290`, `max=325`

In [ ]:
# Visualize LST
lst_vis = {
    'min': 290,
    'max': 325,
    'palette': ['blue', 'yellow', 'red']
}
Map.addLayer(lst_2013, lst_vis, 'LST 2013')
Map.addLayer(lst_2024, lst_vis, 'LST 2024')
Map

## Step 7: (Optional) Analyze statistics by land cover class

In [ ]:
# Optionally compute zonal statistics

## Step 8: (Optional - Easter Egg :)) Generate your own Land Cover Classification using TerraTorch and foundation models*

Based on the example/tutorial: https://aiforgood.itu.int/event/workshop-earth-observation-foundation-models-with-prithvi-eo-2-0-and-terratorch/

*to earn 5.0 grade that will make a great impact on your final grade